In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

from pymilvus import MilvusClient, CollectionSchema
from milvus_model.dense.cohere import CohereEmbeddingFunction

In [ ]:
_ = load_dotenv(find_dotenv())
milvus_uri = os.getenv("MILVUS_URI")
cohere_api_key = os.getenv("COHERE_API_KEY")

In [ ]:
client = MilvusClient(uri=milvus_uri)

Collectionの作成

In [ ]:
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=1024,
)

データの準備

In [ ]:
docs = [
    "人工知能は1956年に学問分野として設立されました。",
    "アラン・チューリングは人工知能の研究を大規模に行った最初の人物です。",
    "ロンドンのマイダ・ベールで生まれたチューリングは、イングランド南部で育ちました。",
]

embedding_fn = CohereEmbeddingFunction(api_key=cohere_api_key, model_name="embed-multilingual-v3.0")

vectors = embedding_fn.encode_documents(documents=docs)

data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"}
    for i in range(len(vectors))
]
print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))

データの格納

In [ ]:
res = client.insert(
    collection_name="demo_collection",
    data=data
)

print(res)

ベクトル検索

In [ ]:
queries = ["アラン・チューリングはだれですか？"]
query_vectors = embedding_fn.encode_queries(queries=queries)

res = client.search(
    collection_name="demo_collection",
    data=query_vectors,
    limit=2,
    output_fields=["text", "subject"],
)

print(res)